In [32]:
# need to mention that the data needs to be uploaded to gee asset store first before running this script

In [33]:
import ee

ee.Initialize()

In [34]:
from cnwi.cnwilib.datautils import ee_features_manifest

features_manifest = ee_features_manifest(
    root='projects/ee-nwrc-geomatics/assets/cnwi/ingested'
)

features_manifest

,type,name,id,updateTime,root,asset_name,data_type,region_id
0,TABLE,projects/ee-nwrc-geomatics/assets/cnwi/ingeste...,projects/ee-nwrc-geomatics/assets/cnwi/ingeste...,2024-01-05T15:20:35.416973Z,ingested,features_122,features,122
1,TABLE,projects/ee-nwrc-geomatics/assets/cnwi/ingeste...,projects/ee-nwrc-geomatics/assets/cnwi/ingeste...,2024-01-05T15:20:57.841998Z,ingested,features_123,features,123
2,TABLE,projects/ee-nwrc-geomatics/assets/cnwi/ingeste...,projects/ee-nwrc-geomatics/assets/cnwi/ingeste...,2024-01-05T15:21:30.916322Z,ingested,features_124,features,124


In [35]:
from pprint import pprint
# features to sample
row = features_manifest.iloc[1]
features = ee.FeatureCollection(row['id'])

pprint(features.first().getInfo())

{'geometry': {'coordinates': [-65.45633494351331, 44.824792284488844],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'ECOREGION_': 123, 'class_name': 1, 'type': 1},
 'type': 'Feature'}


In [36]:
# load a module from path
import importlib

module = importlib.import_module('payload')
payload = module.Payload


In [37]:
from cnwi.cnwilib.image_collection import DataCubeProcessor, S1Processor, ALOSProcessor
# process the data cube

def image_processing(features, payload) -> ee.Image:
    
    # Data Cube Processing
    dc_id = payload.s2
    dc_col = ee.ImageCollection(dc_id).filterBounds(features)
    dc_proc = DataCubeProcessor(dc_col).process().collection

    # S1 Processing
    s1_col = ee.ImageCollection(module.Payload.s1).filterBounds(features)
    s1_proc = S1Processor(s1_col).process().collection

    alos_proc = ALOSProcessor(
        start_date='2018',
        end_date='2021',
        region=features,
    ).process().collection
    
    # combine all the images
    return ee.Image.cat(dc_proc, s1_proc, alos_proc)

image = image_processing(features, payload)
bands = image.bandNames().getInfo()

with open('bands.txt', 'w') as f:
    f.write('\n'.join(bands))

In [38]:
for _, row in features_manifest.iterrows():
    if not ee.data._credentials:
        ee.Initialize()

    features = ee.FeatureCollection(row['id'])
    
    image = image_processing(features, payload)
    
    sample = image.sampleRegions(
        collection=features,
        tileScale=16,
        scale=10,
        geometries=True,
    )
    asset_id = f"projects/ee-nwrc-geomatics/assets/cnwi/samples/sampled_{row['region_id']}"
    task = ee.batch.Export.table.toAsset(
        assetId=asset_id,
        description="",
        collection=sample,
    )
    task.start()
    ee.Reset()